# Домашнее задание №2

**

## Часть 1. ML workflow (**всего 5 баллов**)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы.

In [27]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [14]:
print(df.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000         

Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [28]:
#your code here
y = df['quality']
X = df.drop('quality', axis=1)
X.shape, y.shape

((1599, 11), (1599,))

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [29]:
#your code here
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.3, random_state=1337)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [31]:
#your code here
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
pred_train = lin_reg.predict(X_train)
pred_test = lin_reg.predict(X_test)
# print(f'Предсказания на трейне', pred_train)
# print(f'Предсказания на тесте', pred_test)


In [32]:
lin_reg.coef_

array([ 8.50202290e-02, -1.18506388e+00, -2.88512149e-01,  2.04447594e-02,
       -1.09083767e+00,  1.73070748e-03, -2.18652050e-03, -4.83960256e+01,
       -1.27470642e-01,  7.67660621e-01,  2.43417610e-01])

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [33]:
#your code here
mse_train = mean_squared_error(y_train, pred_train)
mse_test = mean_squared_error(y_test, pred_test)
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f'mse на трейне', mse_train)
print(f'mse на тесте', mse_test)
print(f'R2 на трейне', r2_train)
print(f'R2 на тесте', r2_test)
# метрики получаются очень низкими

mse на трейне 0.4301205007984489
mse на тесте 0.4167836393917615
R2 на трейне 0.3293240874798029
R2 на тесте 0.3640080126452774


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [34]:
#your code here
r2_mean = np.mean(cross_val_score(lin_reg, X, y, scoring='r2', cv=5))
print(f'Среднее качество модели', r2_mean)

Среднее качество модели 0.29004162884219686


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1.

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [40]:
%%time
from sklearn.linear_model import Lasso
alphas = {'alpha': np.arange(0.1,1.2,0.1)}
lasso = Lasso()
grid = GridSearchCV(lasso, param_grid=alphas, scoring='r2', cv=5, return_train_score=True)
grid.fit(X_train, y_train)
print('best score:', grid.best_score_)
print('best params:', grid.best_params_)
#your code here

best score: 0.18529989437792668
best params: {'alpha': 0.1}
CPU times: user 584 ms, sys: 697 µs, total: 584 ms
Wall time: 755 ms


### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [41]:
lr_lasso = Lasso(alpha=0.1).fit(X_train, y_train)
best_estimator = grid.best_estimator_
best_score = grid.best_score_

print(f"Лучший estimator: {best_estimator}")
print(f"Лучший score на тренировочных данных: {best_score}")

Лучший estimator: Lasso(alpha=0.1)
Лучший score на тренировочных данных: 0.18529989437792668


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [42]:
#your code here
pred_lasso = best_estimator.predict(X_test)
r2_lasso = r2_score(y_test, pred_lasso)
print(f'r2 для теста', r2_lasso)

r2 для теста 0.24041587145749266


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [43]:
from sklearn.preprocessing import StandardScaler

lin_pipeline = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(degree=2)), ('linear', lr_lasso)])

r2_pipe = np.mean(cross_val_score(lin_pipeline, X, y, scoring='r2', cv=5))
r2_pipe
#your code here

0.24057689125481424

### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [12]:
#your code here
lin_pipeline.fit(X_train, y_train)

pipe_pred_train = lin_pipeline.predict(X_train)
pipe_pred_test = lin_pipeline.predict(X_test)

mse_train_pipe = mean_squared_error(y_train, pipe_pred_train)
mse_test_pipe = mean_squared_error(y_test, pipe_pred_test)
r2_train_pipe = r2_score(y_train, pipe_pred_train)
r2_test_pipe = r2_score(y_test, pipe_pred_test)
print(f'mse на трейне', mse_train_pipe)
print(f'mse на тесте', mse_test_pipe)
print(f'R2 на трейне', r2_train_pipe)
print(f'R2 на тесте', r2_test_pipe)

mse на трейне 0.46189739822677167
mse на тесте 0.46539063663508384
R2 на трейне 0.27977518283508396
R2 на тесте 0.2898360494145701


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

1) Качество у модели довольно среднее, мсе=0.41 не самый плохой результат на наших данных, однако r^2=0.36 плохой результат, так как всего 36% дисперсии объясняются моделью
Модель получается не переобученной, так как разница метрик на тесте и на трейне очень мала и коэффициенты близки к 0

2) Так как модель не была переобучена, то добавление лассо не внесла сильных изменений в положительную сторону, а наоборот сделала модель хуже, r2=0.24 - оцень низко

3) Качество выросло относительно модели лассо, r2 стал 0.28. Добавление полинома тоже ГЛОБАЛЬНО не влияет, но результат вырос, это приятно

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [44]:
#your code here
ss = StandardScaler()
ss.fit(X_train)
X_train_ss = ss.transform(X_train)
X_test_ss = ss.transform(X_test)

In [46]:
lin_ss = Lasso()
lin_ss.fit(X_train_ss, y_train)
params = {'alpha': np.arange(1e-14, 1.3, 0.01)}
grid_ss = GridSearchCV(lin_ss, param_grid=params, cv=5)
grid_ss.fit(X_train_ss, y_train)
print('best score:', grid_ss.best_score_)
print('best params:', grid_ss.best_params_)

best score: 0.28586692284759324
best params: {'alpha': 0.030000000000009998}


In [48]:
lin_ss = Lasso(alpha=1e-14)
lin_ss.fit(X_train_ss, y_train)
pred = lin_ss.predict(X_test_ss)
r2ss = r2_score(y_test, pred)
r2ss

0.3640080126452755

в данном случае я сделал масштбирование данных, так как у нас данные очень разношёрстные, то важно было натрейнить скейлер, чтобы он подогнался под наши данные
так же у Лассо нет особо гиперпараметров, которые можно подобрать, поэтому тут только можно "играться" с данными и альфой

## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [10]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [11]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`.

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [12]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


#### Способ 2

In [13]:
'''
     Differently to `.target.mean()` function `transform`
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection.

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях.

In [18]:
import warnings

warnings.filterwarnings('ignore')
# в ячейке ниже вылезают предупреждения, что вношу изменения в нескопированный дф
# пробовал поставить .copy(), но всё равно продолжает ругаться, обычно помогает

In [26]:
# YOUR CODE GOES HERE
%%time
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=False)
kf.get_n_splits(all_data)

for train_index, test_index in kf.split(all_data):
    train, test = all_data.iloc[train_index], all_data.iloc[test_index]
    item_id_target_mean = train.groupby('item_id').target.mean().copy()
    all_data['item_target_enc'].iloc[test_index] = test['item_id'].map(item_id_target_mean)
all_data['item_target_enc'].fillna(0.3343, inplace=True)
encoded_feature = all_data['item_target_enc'].values
# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4164590712798811


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`.

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [21]:
# YOUR CODE GOES HERE
%%time
encoded_feature = all_data.groupby('item_id')['target'].transform(lambda x: (x.sum() - x) / (x.count() - 1))
encoded_feature = encoded_feature.values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
# ну кстати ждать всё равно долго получается :/

0.4803848311293002
CPU times: user 16.6 s, sys: 850 ms, total: 17.5 s
Wall time: 17.8 s


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [25]:
# YOUR CODE GOES HERE
%%time
mean_target = all_data.groupby('item_id').target.mean()
nrows = all_data.groupby('item_id').size()
all_data['item_target_enc'] = all_data['item_id'].map((mean_target * nrows + 0.3343 * 100) / (nrows + 100))
encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4818198797097264
CPU times: user 823 ms, sys: 270 ms, total: 1.09 s
Wall time: 1.16 s


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [24]:
# YOUR CODE GOES HERE
%%time
cumulative_sum = all_data.groupby('item_id').target.cumsum() - all_data.target
cumulative_count = all_data.groupby('item_id').cumcount()
all_data['item_target_enc'] = cumulative_sum / cumulative_count
all_data['item_target_enc'].fillna(0.3343, inplace=True)
encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.5025245211081697
CPU times: user 1.52 s, sys: 633 ms, total: 2.15 s
Wall time: 2.1 s


Ожидаемый ответ 0.5025